In [41]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import time

In [42]:
# Code Generation
nr_codewords = int(1e6)
bits_info = torch.randint(2, (nr_codewords, 4), dtype=torch.int)

print(bits_info)

tensor([[1, 0, 0, 0],
        [0, 1, 1, 0],
        [1, 0, 1, 0],
        ...,
        [1, 0, 1, 1],
        [1, 1, 1, 1],
        [0, 1, 1, 1]], dtype=torch.int32)


Hamming(7,4) Encoder

In [43]:
class hamming_encode(torch.nn.Module):
    def __init__(self):
        """
            Use Hamming(7,4) to encode the data.
    
        Args:
            data: data received from the Hamming(7,4) encoder(Tensor)
            generator matrix: generate the parity code
    
        Returns:
            encoded data: 4 bits original info with 3 parity code.
        """
        super(hamming_encode, self).__init__()

        # Define the generator matrix for Hamming(7,4)
        self.generator_matrix = torch.tensor([
            [1, 0, 0, 0],
            [0, 1, 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1],
            [1, 1, 0, 1],
            [1, 0, 1, 1],
            [0, 1, 1, 1],
        ], dtype=torch.int)

    def forward(self, input_data):
        # Ensure input_data has shape (batch_size, 4)
        assert input_data.size(1) == self.generator_matrix.shape[1], "Input data must have same generator matrix row number bits."

        # Perform matrix multiplication to encode the data
        encoded_data = torch.matmul(input_data, self.generator_matrix.t()) % 2

        return encoded_data

In [44]:
encoder = hamming_encode()
encoded_codeword = encoder(bits_info)
print(encoded_codeword)

tensor([[1, 0, 0,  ..., 1, 1, 0],
        [0, 1, 1,  ..., 1, 1, 0],
        [1, 0, 1,  ..., 1, 0, 1],
        ...,
        [1, 0, 1,  ..., 0, 1, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [0, 1, 1,  ..., 0, 0, 1]], dtype=torch.int32)


BPSK Modulator + Noise

In [45]:
class bpsk_modulator(torch.nn.Module):
    def __init__(self):
        """
        Use BPSK to compress the data, which is easily to transmit.

        Args:
            codeword: data received from the Hamming(7,4) encoder(Tensor)
    
        Returns:
            data: Tensor contain all data modulated and add noise
        """
        super(bpsk_modulator, self).__init__()
        
    def forward(self, codeword, snr_dB):
        
        # data = torch.tensor(data, dtype=float)
        data = codeword.to(dtype=torch.float)
    
        for i in range(data.shape[0]):
            bits = data[i]
            bits = 2 * bits - 1
        
            # Add Gaussian noise to the signal
            noise_power = torch.tensor(10**(snr_dB / 10))
            noise = torch.sqrt(1/(2*noise_power)) * torch.randn(len(bits))
            noised_signal = bits + noise
            # noised_signal = bits
            data[i] = noised_signal
    
       
        return data

In [46]:
snr_dB = 15  # Signal-to-noise ratio in dB

# Modulate the signal
modulator = bpsk_modulator()
modulated_noise_signal = modulator(encoded_codeword, snr_dB)
print(modulated_noise_signal)

tensor([[ 1.0631, -0.9806, -1.0718,  ...,  1.0883,  0.9422, -1.1346],
        [-1.0285,  1.0144,  1.0871,  ...,  0.9736,  1.0569, -1.2081],
        [ 0.9229, -1.0786,  0.9848,  ...,  1.0077, -0.8773,  0.9653],
        ...,
        [ 0.9932, -0.8789,  1.1574,  ..., -1.1213,  1.1482, -1.1460],
        [ 0.8654,  0.9372,  1.1239,  ...,  0.9685,  1.0524,  1.0395],
        [-1.0688,  0.8893,  0.7677,  ..., -0.9065, -1.2766,  0.8497]])


LLR Log-likelihood
y = s + n
Assuming that \( s \) is equally likely to be 0 or 1, and \( n \) is Gaussian with zero mean and variance \( N_0/2 \), where \( N_0 \) is the noise power spectral density.


In [47]:
def llr(signal, snr):
    """
    Calculate Log Likelihood Ratio (LLR) for a simple binary symmetric channel.

    Args:
        signal (torch.Tensor): Received signal from BPSK.
        noise_std (float): Standard deviation of the noise.

    Returns:
        llr: Log Likelihood Ratio (LLR) values.
    """
    
    # Assuming Binary Phase Shift Keying (BPSK) modulation
    noise_std = torch.sqrt(torch.tensor(10**(snr / 10)))

    # Calculate the LLR
    llr = 2 * signal * noise_std

    # return llr_values, llr
    return llr


In [48]:
llr_output = llr(modulated_noise_signal, snr_dB)
print("LLR values:", llr_output)

LLR values: tensor([[ 11.9567, -11.0289, -12.0539,  ...,  12.2401,  10.5968, -12.7609],
        [-11.5679,  11.4090,  12.2266,  ...,  10.9500,  11.8864, -13.5871],
        [ 10.3796, -12.1307,  11.0754,  ...,  11.3330,  -9.8671,  10.8563],
        ...,
        [ 11.1708,  -9.8846,  13.0169,  ..., -12.6111,  12.9137, -12.8891],
        [  9.7325,  10.5407,  12.6404,  ...,  10.8927,  11.8359,  11.6916],
        [-12.0209,  10.0022,   8.6341,  ..., -10.1957, -14.3582,   9.5562]])


LDPC Decoder

Strange behavior recording:
In 100 7-bit codewords, the speed on MPS(GPU) is slower than CPU. Reason unknown.

In [49]:
class LDPCBeliefPropagation(torch.nn.Module):
    def __init__(self, H):
        """
        LDPC Belief Propagation.
    
        Args:
            H: Low density parity code for building tanner graph.
            llr: Log Likelihood Ratio (LLR) values. Only for 7-bit codeword.
    
        Returns:
            estimated_bits: the output result from belief propagation.
        """
        
        super(LDPCBeliefPropagation, self).__init__()
        self.H = H
        self.num_check_nodes, self.num_variable_nodes = H.shape

        # Initialize messages
        self.messages_v_to_c = torch.ones((self.num_variable_nodes, self.num_check_nodes),dtype=torch.float)
        self.messages_c_to_v = torch.zeros((self.num_check_nodes, self.num_variable_nodes),dtype=torch.float)

    def forward(self, llr, max_iter):
        for iteration in range(max_iter):
            # Variable to check node messages
            for i in range(self.num_variable_nodes):
                for j in range(self.num_check_nodes):
                    # Compute messages from variable to check nodes
                    connected_checks = self.H[j, :] == 1
                    product = torch.prod(torch.tanh(0.5 * self.messages_v_to_c[connected_checks, j]))
                    self.messages_v_to_c[i, j] = torch.sign(llr[i]) * product

            # Check to variable node messages
            for i in range(self.num_check_nodes):
                for j in range(self.num_variable_nodes):
                    # Compute messages from check to variable nodes
                    connected_vars = self.H[:, j] == 1
                    sum_msgs = torch.sum(self.messages_c_to_v[connected_vars, i]) - self.messages_v_to_c[j, i]
                    self.messages_c_to_v[i, j] = 2 * torch.atan(torch.exp(0.5 * sum_msgs))

        # Calculate the final estimated bits and only take first four bits
        estimated_bits = torch.sign(llr) * torch.prod(torch.tanh(0.5 * self.messages_c_to_v), dim=0)
        estimated_bits = torch.where(estimated_bits>0, torch.tensor(1), torch.tensor(0))
        estimated_bits = estimated_bits[0:4]
        estimated_bits = estimated_bits.to(dtype=torch.int)


        return estimated_bits
    

In [53]:
# Define LDPC parameters
H = torch.tensor([ [1, 1, 1, 0, 0, 0, 0],
                   [0, 0, 1, 1, 1, 0, 0],
                   [0, 1, 0, 0, 1, 1, 0],
                   [1, 0, 0, 1, 0, 0, 1],])
iter = 10
ldpc_bp = LDPCBeliefPropagation(H)
n = 2000 # For n iteration, print time

# Store the final result from LDPC
tensor_size = torch.Size([llr_output.shape[0], 4])
final_result = torch.zeros(tensor_size)

#loop all the llr and get result.
for i in range(llr_output.shape[0]):
    start_time = time.time()
    
    bp_data = llr_output[i]
    estimated_bits = ldpc_bp(bp_data, iter)
    final_result[i] = estimated_bits
    
    end_time = time.time()
    iteration_time = (end_time - start_time)*n
    
    if i>0 and i % n == 0:
        print(f"Procedure {i}, Time taken for {n} 7-bit codeword: {iteration_time} seconds")

print(final_result)

Procedure 2000, Time taken for 2000 4-bit codeword: 24.868011474609375 seconds
Procedure 4000, Time taken for 2000 4-bit codeword: 25.191783905029297 seconds
Procedure 6000, Time taken for 2000 4-bit codeword: 26.350021362304688 seconds
Procedure 8000, Time taken for 2000 4-bit codeword: 24.625778198242188 seconds
Procedure 10000, Time taken for 2000 4-bit codeword: 24.69778060913086 seconds
Procedure 12000, Time taken for 2000 4-bit codeword: 24.89614486694336 seconds
Procedure 14000, Time taken for 2000 4-bit codeword: 24.951457977294922 seconds
Procedure 16000, Time taken for 2000 4-bit codeword: 25.205612182617188 seconds
Procedure 18000, Time taken for 2000 4-bit codeword: 24.741649627685547 seconds
Procedure 20000, Time taken for 2000 4-bit codeword: 24.86419677734375 seconds
Procedure 22000, Time taken for 2000 4-bit codeword: 25.529861450195312 seconds
Procedure 24000, Time taken for 2000 4-bit codeword: 24.675846099853516 seconds
Procedure 26000, Time taken for 2000 4-bit code

Comparation and Plot

In [56]:
def calculate_ber(transmitted_bits, origin_bits):
    # Ensure that both tensors have the same shape
    assert transmitted_bits.shape == origin_bits.shape, "Shapes of transmitted and received bits must be the same."

    # Calculate the bit errors
    errors = (transmitted_bits != origin_bits).sum().item()
    print(errors)

    # Calculate the Bit Error Rate (BER)
    ber = errors / transmitted_bits.numel()

    return ber

In [62]:
# Describe the data:
# bits_info: original signal
decoded_bits = final_result #output from Maximum Likelihood
# decoded_bits = llr_output # Output from log-likelihood
# decoded_bits = 

ber = calculate_ber(decoded_bits, bits_info)
# print(ber)

print(final_result)


0
tensor([[1., 0., 0., 0.],
        [0., 1., 1., 0.],
        [1., 0., 1., 0.],
        ...,
        [1., 0., 1., 1.],
        [1., 1., 1., 1.],
        [0., 1., 1., 1.]])
